In [1]:
import pandas as pd
import numpy as np

### Собственная реализация градиентного спуска для задачи линейной регрессии

In [174]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import StandardScaler

class MyLinearRegression(BaseEstimator, RegressorMixin):

    def __init__(self, fit_intercept = True):
        self.fit_intercept = fit_intercept
    
    @staticmethod
    def mse(y_true, y_predictions):
        return np.mean((y_true - y_predictions) ** 2)
    
    def fit(self, X, y, learning_rate = 1e-2, loss_func = mse, tol = 1e-5, max_iter = 10000, mu = 0.9):
        n_objects, self.n_features = X.shape
        # изначальные веса
        self.coef = np.random.rand(self.n_features)
        self.intercept = 0
        # изначальный loss
        predictions = self.predict(X)
        self.loss = loss_func(y, predictions)
        self.last_loss = 1e6

        # импульсы
        v_coef = np.zeros_like(self.coef)
        v_intercept = 0
        
        def loss_difference(curr_loss, last_loss):
            return abs(last_loss - curr_loss) / max(1, last_loss)
        
        while loss_difference(self.loss, self.last_loss)>tol and max_iter>0:           
            # из текущей точки по инерции смотрим вперед
            coef_lookahead = self.coef + mu * v_coef
            intercept_lookahead = self.intercept + mu * v_intercept
            
            # из будущей точки считаем градиент
            predictions = self.predict(X, coef_lookahead, intercept_lookahead)
            dw = (2.0 / n_objects) * (X.T @ (predictions - y))
            db = (2.0 / n_objects) * np.sum(predictions - y)

            # обновляем импульс
            v_coef = mu * v_coef - learning_rate * dw
            v_intercept = mu * v_intercept - learning_rate * db

            # шагаем из изначальной точки по новому импульсу
            self.coef += v_coef
            self.intercept += v_intercept

            # пересчитываем лоссы
            self.last_loss = self.loss
            predictions = self.predict(X)
            self.loss = loss_func(y, predictions)
            max_iter-=1
        return self
        
    def predict(self, X, coef = None, intercept = None):
        if coef is None:
            coef = self.coef
        if intercept is None:
            intercept = self.intercept
        return X @ coef + intercept

### Сравнение по качеству

### Моя реализация

In [175]:
from sklearn.datasets import load_diabetes, fetch_california_housing
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target


In [176]:
my_lr = MyLinearRegression()

In [177]:
from sklearn.model_selection import cross_val_score

print(f'Mean RMSE: {np.mean(np.sqrt(-cross_val_score(my_lr, X, y, cv = 7, scoring = 'neg_mean_squared_error')))}')

Mean RMSE: 60.24531167841359


### Встроенная линейная регрессия

In [168]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
pipe = Pipeline([('LR', LinearRegression())])

In [169]:
print(f'Mean RMSE: {np.mean(np.sqrt(-cross_val_score(pipe, X, y, cv = 7, scoring = 'neg_mean_squared_error')))}')

Mean RMSE: 54.760886101733306
